In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O ''(e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/.gitattributes
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00002-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/gener

In [2]:
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes --no-cache-dir
import bitsandbytes
print(bitsandbytes.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 246.4 MB/s eta 0:00:00a 0:00:01
0.45.2


In [35]:
# !pip install bitsandbytes==0.43.1

In [3]:
from pprint import pprint
import wandb
import math
import datasets

In [4]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)  # Optional: Check memory usage
!pip cache purge


Files removed: 0


In [5]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from transformers import DataCollatorWithPadding
from transformers import LlamaConfig, LlamaForCausalLM, LlamaForSequenceClassification
from transformers import TrainingArguments, Trainer, BitsAndBytesConfig

In [7]:
import transformers, peft, accelerate
import torch

In [8]:
import pandas as pd
df_train = pd.read_csv('/kaggle/input/multi-lingual-sentiment-analysis/train.csv')
df_test = pd.read_csv('/kaggle/input/multi-lingual-sentiment-analysis/test.csv')
model_path = '/kaggle/input/llama-3.1/transformers/8b-instruct/2'
# df_train["label"] = df_train["label"].map({"Positive": 1, "Negative": 0})
df_train.label

0      Positive
1      Positive
2      Positive
3      Positive
4      Negative
         ...   
995    Negative
996    Negative
997    Negative
998    Negative
999    Negative
Name: label, Length: 1000, dtype: object

In [9]:
df_train.label

0      Positive
1      Positive
2      Positive
3      Positive
4      Negative
         ...   
995    Negative
996    Negative
997    Negative
998    Negative
999    Negative
Name: label, Length: 1000, dtype: object

In [10]:
print(df_train.columns)
from datasets import Dataset, DatasetDict


Index(['ID', 'sentence', 'label', 'language'], dtype='object')


In [11]:
dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)

dataset = DatasetDict({
    "train": dataset_train,
    "test": dataset_test
})


In [12]:
!pip install peft accelerate

In [13]:
dataset['train'].features

{'ID': Value(dtype='int64', id=None),
 'sentence': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None)}

In [14]:
def tokenize(eg):
    eg = tokenizer(eg['sentence'], padding=False,truncation=True)
    return eg

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.model_max_length=512

In [16]:
dataset['train'].features

{'ID': Value(dtype='int64', id=None),
 'sentence': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'language': Value(dtype='string', id=None)}

In [17]:
tokenized_ds = dataset['train'].map(tokenize, batched=True, remove_columns=['sentence','ID','language'])
tokenized_test_ds = dataset['test'].map(tokenize, batched=True, remove_columns=['sentence','ID','language'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [50]:
# !pip install -q --no-deps xformers trl peft accelerate 
# !pip install --no-cache-dir --force-reinstall bitsandbytes


In [18]:
print(tokenized_ds)

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 1000
})


In [19]:
ds_split = tokenized_ds.train_test_split(test_size=0.1, seed=42)
print(ds_split)

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})


In [20]:
ds_split

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [21]:
data_collator = DataCollatorWithPadding(tokenizer, padding=True)

In [22]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)


In [23]:
from transformers import AutoModelForCausalLM

In [24]:
model = AutoModelForCausalLM.from_pretrained(model_path ,
                                                           # num_labels=2,
                                                           quantization_config=quantization_config,
                                                           # device_map="auto",
                                                         
                                                           pad_token_id = tokenizer.eos_token_id)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [58]:
# model = AutoModelForSequenceClassification.from_pretrained(model_path ,
#                                                            num_labels=2,
#                                                            quantization_config=quantization_config,
#                                                            # device_map="auto",
                                                         
#                                                            pad_token_id = tokenizer.eos_token_id)


In [59]:
# model.config.id2label = {0:"NEGATIVE",1:"POSITIVE"}

In [25]:
for name,param in model.named_parameters():
    if name!='score.weight':
        param.requires_grad = False
    print(name, param.requires_grad)

model.embed_tokens.weight False
model.layers.0.self_attn.q_proj.weight False
model.layers.0.self_attn.k_proj.weight False
model.layers.0.self_attn.v_proj.weight False
model.layers.0.self_attn.o_proj.weight False
model.layers.0.mlp.gate_proj.weight False
model.layers.0.mlp.up_proj.weight False
model.layers.0.mlp.down_proj.weight False
model.layers.0.input_layernorm.weight False
model.layers.0.post_attention_layernorm.weight False
model.layers.1.self_attn.q_proj.weight False
model.layers.1.self_attn.k_proj.weight False
model.layers.1.self_attn.v_proj.weight False
model.layers.1.self_attn.o_proj.weight False
model.layers.1.mlp.gate_proj.weight False
model.layers.1.mlp.up_proj.weight False
model.layers.1.mlp.down_proj.weight False
model.layers.1.input_layernorm.weight False
model.layers.1.post_attention_layernorm.weight False
model.layers.2.self_attn.q_proj.weight False
model.layers.2.self_attn.k_proj.weight False
model.layers.2.self_attn.v_proj.weight False
model.layers.2.self_attn.o_proj

In [26]:
num_para = 0
for param in model.parameters():
    if param.requires_grad:
        num_para += param.numel()
print(f'Number of Parameters:{ num_para}')

Number of Parameters:0


In [27]:
from transformers import TextGenerationPipeline
classifier = TextGenerationPipeline(model=model,
                                       tokenizer=tokenizer,
                                       framework='pt',
                                       task="text-generation",
                                       # device = "cuda"
                                       )

Device set to use cuda:0


In [28]:
text = "LLama Can you predict if the movie is good or bad. Good review will return postive. Now for this statement: 'The movie is worst'"
prediction = classifier(text)
print(prediction)

[{'generated_text': "LLama Can you predict if the movie is good or bad. Good review will return postive. Now for this statement: 'The movie is worst' - Can you predict the good review or bad review.\nNo, I can't predict whether the review"}]


In [29]:
dataset_test['sentence']

['1120 mAh, ਓਵਰਚਾਰਜਿੰਗ ਦੀ ਸੁਰੱਖਿਆ',
 'તે સઘન મોઇશ્ચરાઇઝિંગ પ્રદાન કરે છે અને સરસ સ્વર્ગીય ફળની સુગંધ આપે છે',
 '1120 ಎಂಎಎಚ್, ಮಿತಿಮೀರಿದ ರಕ್ಷಣೆ',
 'ভাৰতত নিৰ্মিত সৰ্বশ্ৰেষ্ঠ পাৰফিউম ব্ৰেণ্ডবোৰৰ এটা। এইটো এটা নিশ্চিতভাৱে পৰীক্ষণীয় সামগ্ৰী। মই বিভিন্ন স্তৰৰ কমলাৰ ফুল, আঙুৰ, কস্তুৰী আৰু জেচমিনৰ গোন্ধ ভাল পাওঁ। সঁচাকৈয়ে আপোনাক গোটেই দিনটো সতেজ কৰি ৰাখে।',
 'میں نے حال ہی میں "انفولڈ" سے ایک ٹیمپلیٹ خریدا ہے لیکن میں اپنے ترمیمی صفحہ تک رسائی یا دیکھنے سے قاصر ہوں۔ یہ کافی پریشان کن ہے، بینک اکاؤنٹ سے رقم کاٹی جا رہی ہے اور ٹیمپلیٹ دکھا رہا ہے، لیکن ترمیم کے موڈ میں ظاہر نہیں ہوتا ہے۔',
 'వెలుపలి నుంచి నియాప్రీన్ మెటీరియల్ కోటింగ్ ఉంటుంది, అయితే లోపలి నుంచి ఎలాంటి కవరింగ్ ఉండదు, అందువల్ల బంప్\u200cల నుంచి లెన్స్\u200cని అంత సమర్థవంతంగా సంరక్షించలేం.',
 '1120 mAh, ஓவர் சார்ஜ் பாதுகாப்பு',
 'ഉഷയിൽ നിന്നുള്ള ഈ പേഴ്സണൽ എയർ കൂളർ വളരെ മിനുസമാർന്ന ബ്ലോവറോടെയാണ് വരുന്നത്, നിങ്ങൾക്ക് ഇത് ഒരു കൂളറായി തോന്നില്ല, മറിച്ച് ഒരു ഫാൻ പോലെയാണ്. ശബ്\u200cദ നില വളരെ കുറവാണ്.',
 'কিন্তু আপনি এই পণ্যের ফ্লাইট/স্থ

In [37]:
i = 0
results = []

# Iterate through each sentence in the dataset
for row in dataset_test['sentence']:
    sentence = row
    
    # Ask the classifier to predict sentiment for the sentence
    prediction = classifier(sentence + "\n predict if the given sentence is negative or positive in sentiment, just give one word answer?", max_new_tokens=100, temperature=0.1, truncation=True)
    print(prediction)  # Print to check what the model returns for each sentence
    predicted_text = prediction[0]['generated_text']
    i = i + 1
    
    # # Check if the word 'positive' or 'Positive' exists in the prediction result
    # if 'positive' in predicted_text.strip().lower(): # .lower() ensures case-insensitive matching
    #     # Fill code for positive sentiment
    #     results.append({'ID': i, 'label': 'Positive'})
    # else:
    #     # Fill code for negative sentiment
    #     results.append({'ID': i, 'label': 'Negative'})

    # Initialize counters for positive and negative occurrences
    positive_count = predicted_text.strip().lower().count('positive')
    negative_count = predicted_text.strip().lower().count('negative')
    
    # Compare counts and assign label based on the higher count
    if positive_count >= negative_count:
        results.append({'ID': i, 'label': 'Positive'})
    else:
        results.append({'ID': i, 'label': 'Negative'})


# Convert results into a DataFrame for easy viewing
result_df = pd.DataFrame(results)

# Print the formatted results
print(result_df)

# If you'd like to save it to CSV, you can uncomment the following line:
# result_df.to_csv('predictions.csv', index=False)


[{'generated_text': '1120 mAh, ਓਵਰਚਾਰਜਿੰਗ ਦੀ ਸੁਰੱਖਿਆ\n predict if the given sentence is negative or positive in sentiment, just give one word answer? \nਸ਼ਬਦ ਸੰਬੰਧਿਤ ਸੁਭਾਵ\nਸ਼ਬਦ ਸੰਬੰਧਿਤ ਸੁਭਾਵ\nਸ਼ਬਦ ਸੰਬੰਧਿਤ ਸੁਭਾਵ\n'}]
[{'generated_text': 'તે સઘન મોઇશ્ચરાઇઝિંગ પ્રદાન કરે છે અને સરસ સ્વર્ગીય ફળની સુગંધ આપે છે\n predict if the given sentence is negative or positive in sentiment, just give one word answer? \n positive\n 1. તે સઘન મોઇશ્ચરાઇઝિંગ પ્રદાન કરે છે અને સરસ સ્વર્ગીય ફળની'}]
[{'generated_text': '1120 ಎಂಎಎಚ್, ಮಿತಿಮೀರಿದ ರಕ್ಷಣೆ\n predict if the given sentence is negative or positive in sentiment, just give one word answer? \nಸಂಕ್ಷಿಪ್ತ ವಿವರಗಳು\nಸಂಪರ್ಕ ವಿಧಾನ\nಪ್ರಶ್ನೆಗಳು\nಪ್ರಯೋಜನಗಳು\n�'}]
[{'generated_text': 'ভাৰতত নিৰ্মিত সৰ্বশ্ৰেষ্ঠ পাৰফিউম ব্ৰেণ্ডবোৰৰ এটা। এইটো এটা নিশ্চিতভাৱে পৰীক্ষণীয় সামগ্ৰী। মই বিভিন্ন স্তৰৰ কমলাৰ ফুল, আঙুৰ, কস্তুৰী আৰু জেচমিনৰ গোন্ধ ভাল পাওঁ। সঁচাকৈয়ে আপোনাক গোটেই দিনটো সতেজ কৰি ৰাখে।\n predict if the given sentence is negative or positive in sentiment, just give o

In [39]:
# Save the DataFrame as 'submission.csv'
result_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
    if 'positive' or 'Positive' or 'POSITIVE' in prediction: 

In [ ]:
dataset_test

In [ ]:
# from transformers import TextClassificationPipeline
# # import pandas as pd

# # Assuming `classifier` is already created like you have it
# # You can load your dataset, assuming it's a HuggingFace Dataset
# # If your dataset is a DataFrame, this can be adjusted accordingly
# # from datasets import load_dataset

# # Example: Load a dataset (you can replace this with your dataset loading code)
# # dataset_test = load_dataset("your_dataset_path_or_name")  # Replace with actual dataset path if needed

# # This assumes the dataset_test is structured with 'ID' and 'sentence' as columns
# # Example for testing
# # dataset_test = pd.DataFrame({
# #     'ID': [1, 2, 3, 4, 5],
# #     'sentence': [
# #         "The movie was amazing!",
# #         "The movie was terrible.",
# #         "Great performances!",
# #         "Worst movie I’ve seen.",
# #         "I loved the story."
# #     ]
# # })

# # Prepare the output
# results = []

# # Iterate through the dataset and classify each sentence
# for index, row in dataset_test.iterrows():
#     sentence = row['sentence']
#     prediction = classifier(sentence)
    
#     # Based on the prediction, classify as Positive or Negative
#     label = "Positive" if prediction[0]['label'] == "POSITIVE" else "Negative"
    
#     # Append the result with ID and label
#     results.append({'ID': row['ID'], 'label': label})

# # Convert results into a DataFrame for easy viewing or saving
# result_df = pd.DataFrame(results)

# # Show the formatted results
# print(result_df)

# # If you want to save to CSV, uncomment the following line:
# # result_df.to_csv('predictions.csv', index=False)


In [ ]:
# import pandas as pd

# # Function to classify and format label
# def predict_label(text, classifier):
#     prediction = classifier(text)
#     label = prediction[0]['label']
#     return 'Positive' if label == 'POSITIVE' else 'Negative'

# # Generate predictions for dataset_test
# submission_data = []
# for example in dataset_test:
#     pred_label = predict_label(example['sentence'], classifier)
#     submission_data.append({'ID': example['ID'], 'label': pred_label})

# # Convert to DataFrame
# submission_df = pd.DataFrame(submission_data)

# # Save as CSV file
# submission_df.to_csv('submission.csv', index=False)

# print("Submission file generated: submission.csv")


In [ ]:
# import pandas as pd

# # Function to classify and format label
# def predict_label(text, classifier):
#     prediction = classifier(text)
#     label = prediction[0]['label']
#     return 'Positive' if label == 'POSITIVE' else 'Negative'

# # Generate predictions for dataset_test
# submission_data = []
# print("Processing predictions...")
# for example in dataset_test:
#     pred_label = predict_label(example['sentence'], classifier)
#     submission_data.append({'ID': example['ID'], 'label': pred_label})
#     print(f"Processed ID: {example['ID']}, Prediction: {pred_label}")

# # Convert to DataFrame
# submission_df = pd.DataFrame(submission_data)

# # Save as CSV file
# submission_df.to_csv('submission.csv', index=False)

# print("Submission file generated: submission.csv")


In [ ]:
df = pd.read_csv('/kaggle/working/submission.csv')
df

In [ ]:
!pip install evaluate

In [ ]:
import numpy as np
import evaluate
metric=evaluate.load('accuracy')
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,  
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=['q_proj','v_proj'],
    bias="none",
    task_type="SEQ_CLS"
)
# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


# Enable gradient checkpointing to reduce memory usage
model.gradient_checkpointing_enable()

In [ ]:
# for name, param in model.named_parameters():
#     if "lora" in name:  # LoRA layers only
#         param.requires_grad = True
#     else:
#         param.requires_grad = False  # Keep quantized layers frozen


In [ ]:
from peft import LoraConfig


In [ ]:
# lora_config = LoraConfig(
#     target_modules=["q_proj", "k_proj"],
#     init_lora_weights=False
# )

# model.add_adapter(lora_config, adapter_name="adapter_1")

In [ ]:
# training_args = TrainingArguments( output_dir='llma32_imdb_ft',
#                                   eval_strategy="steps",
#                                   eval_steps=100,
#                                   num_train_epochs=1,
#                                   per_device_train_batch_size=12,
#                                   per_device_eval_batch_size=12,
#                                   bf16=True,
#                                   fp16=False,
#                                   tf32=False,
#                                   gradient_accumulation_steps=1,
#                                   adam_beta1=0.9,
#                                   adam_beta2=0.999,
#                                   learning_rate=2e-5,
#                                   weight_decay=0.01,
#                                   logging_dir='logs',
#                                   logging_strategy="steps",
#                                   logging_steps = 100,
#                                   save_steps=100,
#                                   save_total_limit=20,
#                                   report_to='none',
#                                 )
# trainer = Trainer(model=model,
#                   args = training_args,
#                  train_dataset=ds_split["train"],
#                  eval_dataset=ds_split["test"],
#                  # compute_metrics=compute_metrics,
#                  data_collator = data_collator)





In [ ]:
# import torch
# torch.cuda.empty_cache()
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
# result = trainer.train()

In [ ]:
# result

In [ ]:
# model.save_to_disk('')

In [ ]:
# test_dataset=pd.read_csv('/kaggle/input/multi-lingual-sentiment-analysis/test.csv')

In [ ]:
# test_dataset = Dataset.

In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from peft import PeftModel

# # Define model and checkpoint path
# base_model_name = "meta-llama/Llama-3-8B-Instruct"  # Update if using another base model
# checkpoint_path = "/kaggle/working/llma32_imdb_ft/checkpoint-100"  # Change to the best checkpoint
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, ignore_mismatched_sizes=True)

# # Load base model
# # tokenizer = AutoTokenizer.from_pretrained(base_model_name)
# # model = AutoModelForSequenceClassification.from_pretrained(base_model_name, torch_dtype=torch.float16, device_map="auto")

# # Load LoRA adapter
# # model = PeftModel.from_pretrained(model, checkpoint_path)

# # Move model to GPU if available
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# print("Model loaded successfully 🎯")


In [ ]:
# print(ds_split["train"][1])
